# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_metadata = "my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cabo San Lucas,22.8909,-109.9124,76.33,78,100,1.77,MX,1623661200
1,1,Oriximiná,-1.7656,-55.8661,73.33,87,71,2.77,BR,1623661200
2,2,George Town,5.4112,100.3354,88.63,64,20,5.37,MY,1623661200
3,3,Sibolga,1.7427,98.7792,86.11,69,92,4.43,ID,1623661200
4,4,Saldanha,-33.0117,17.9442,57.94,72,32,2.98,ZA,1623661200


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
locations = weatherdf[["Latitude", "Longitude"]].astype(float)
hum_rate = weatherdf["Humidity"].astype(float)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,Middelkerke,51.1558,2.8418,73.20,54,0,5.84,BE,1623661200
143,143,Masvingo,-20.0637,30.8277,70.27,52,0,6.91,ZW,1623661200
156,156,Yuci,37.6803,112.7319,73.89,73,0,3.20,CN,1623661200
204,204,Saint-François,46.4154,3.9054,73.90,45,0,1.57,FR,1623661200
306,306,Saravan,39.7196,45.6483,77.65,30,0,7.56,AM,1623661200


<Figure size 432x288 with 0 Axes>

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
24,Middelkerke,BE,51.1558,2.8418
143,Masvingo,ZW,-20.0637,30.8277
156,Yuci,CN,37.6803,112.7319
204,Saint-François,FR,46.4154,3.9054
306,Saravan,AM,39.7196,45.6483


In [11]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    results
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 24: Middelkerke.
Missing field/result... skipping.
------------
Retrieving Results for Index 143: Masvingo.
Missing field/result... skipping.
------------
Retrieving Results for Index 156: Yuci.
Missing field/result... skipping.
------------
Retrieving Results for Index 204: Saint-François.
Missing field/result... skipping.
------------
Retrieving Results for Index 306: Saravan.
Missing field/result... skipping.
------------
Retrieving Results for Index 337: Kasane.
Missing field/result... skipping.
------------
Retrieving Results for Index 391: Saint-Pierre.
Missing field/result... skipping.
------------
Retrieving Results for Index 479: Otjimbingwe.
Missing field/result... skipping.
------------
Retrieving Results for Index 489: Ampanihy.
Missing field/result... skipping.
------------
Retrieving Results for Index 505: Sirte.
Missing field/result... skipping.
------------


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [39]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig

# Display figure


NameError: name 'locations2' is not defined